<a href="https://colab.research.google.com/github/steffenvogler/LLM2GPS/blob/main/WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain geopy google-api-python-client google-cloud-aiplatform

In [8]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Getting started**

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

the project must be present under https://console.cloud.google.com/ using the same account as used in above authentication

In [2]:
import vertexai

PROJECT_ID = ""  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="europe-west4")

In [3]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.330
Vertex AI SDK version: 1.36.0


In [4]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI()

check this for more details and code snipptes: [Link to exmaple notebook](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/intro_langchain_palm_api.ipynb#scrollTo=eVpPcvsrkzCk)

In [24]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [93]:
template = """Your job is to guess the geographic loaction based on the description and keywords.
% USER input
{user_input}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Given a location, provide a structured output that starts  with the country, federal state, then administrative district or city (if applicable). Everything separated by a whitespace. Only provide a single line of output. Never output any additional text. If there are more than one places that match the description or if you are not sure, add an asterisk sign ad the end of the response.
% JSON
{user_json}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_json"], template=template)

# Holds my 'json' chain
json_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[location_chain, json_chain], verbose=False)

In [96]:
from geopy.geocoders import Nominatim
from geopy import distance
geolocator = Nominatim(user_agent="LLM2GPS")

In [102]:
#query = "Berlin"
#query = "Spreewald"
#query = "Zugspitze"
#query = "Mount Ararat"
#query = "Sandwich islands"
query = "Gjoa Haven"
improved_query = str(query + " " + overall_chain.run(query))
location = geolocator.geocode(improved_query)
print(improved_query)

def response_processining(location):
  if location is None:
    print("No location found")
    improved_query = str(overall_chain.run(query)+" "+query)
    print(improved_query)
    location2 = geolocator.geocode(improved_query)
    response_processining(location2)
  else:
    print((location.latitude, location.longitude))
    print(location.address)
    print(location.raw['boundingbox'])
    print(location.raw)
    xy0 = (location.raw['boundingbox'][0], location.raw['boundingbox'][2])
    xy1 = (location.raw['boundingbox'][1], location.raw['boundingbox'][3])
    print("Great circle distance in km: {}".format(round(distance.great_circle(xy0, xy1).km,2)))

response_processining(location)

Gjoa Haven 
(68.64252755, -95.87829702808895)
Gjoa Haven, ᕿᑎᕐᒥᐅᑦ Kitikmeot Region, ᓄᓇᕗᑦ Nunavut, Canada
['68.5865275', '68.6982928', '-95.9999992', '-95.7502398']
{'place_id': 267623347, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 437472437, 'lat': '68.64252755', 'lon': '-95.87829702808895', 'class': 'place', 'type': 'town', 'place_rank': 18, 'importance': 0.38127088050974167, 'addresstype': 'town', 'name': 'Gjoa Haven', 'display_name': 'Gjoa Haven, ᕿᑎᕐᒥᐅᑦ Kitikmeot Region, ᓄᓇᕗᑦ Nunavut, Canada', 'boundingbox': ['68.5865275', '68.6982928', '-95.9999992', '-95.7502398']}
Great circle distance in km: 16.02


In [34]:
import math

def haversine(lat1, lon1, lat2, lon2):
  """Calculates the great circle distance between two points on a sphere.

  Args:
    lat1: The latitude of the first point in degrees.
    lon1: The longitude of the first point in degrees.
    lat2: The latitude of the second point in degrees.
    lon2: The longitude of the second point in degrees.

  Returns:
    The great circle distance between the two points in kilometers.
  """

  R = 6371  # Earth's radius in kilometers

  dlat = math.radians(float(lat2) - float(lat1))
  dlon = math.radians(float(lon2) - float(lon1))
  a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(float(lat1))) * math.cos(math.radians(float(lat2))) * math.sin(dlon / 2) ** 2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  d = R * c

  return d

In [56]:
# TODO: of 'type' = 'peak' a width/height calculation does not make sense (may be use national park instead?)
# TODO: check the 'type' provided by Nominatim geocoders
east_west_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][0], location.raw['boundingbox'][3])
north_south_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][1], location.raw['boundingbox'][2])
print("north_south_distance: {} and east-west-distance: {} (all in km)".format(north_south_dist, east_west_dist))

north_south_distance: 0.1192454393342144 and east-west-distance: 0.23887746336239363 (all in km)


In [79]:
location = geolocator.geocode("New zealand; Mt Cook")
print(location)

Aoraki / Mount Cook, Mackenzie District, Canterbury, New Zealand / Aotearoa
